# AutoGen のトレースを Azure AI Foundry で永続化

## 1. OpenAI SDK をインストルメント化する
まずシンプルに OpenAI SDK を使用して開発する場合は、コードをインストルメント化して、トレースが Azure AI Foundry に送信されるようにすることができます。 

環境に `azure-ai-projects`、 `azure-monitor-opentelemetry`、および `opentelemetry-instrumentation-openai-v2` をインストールします。 pip の使用例を次に示します。


In [ ]:
!pip install azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai

## OpenAIInstrumentor()を使用して OpenAI SDK をインストルメント化

In [ ]:
from openai import AzureOpenAI

from opentelemetry.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument()


## プロジェクトでトレースを有効にする

https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-application#enable-tracing-in-your-project

## Azure Application Insights リソースへの接続
プロジェクトに関連付けられている Azure Application Insights リソースへの接続文字列を取得します。 次の行では、認証に Microsoft Entra ID を使用する必要がある Azure AI Project クライアントを使用します。

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint="https://<YourFoundryService>.services.ai.azure.com/api/projects/xxx",
)

connection_string = project_client.telemetry.get_connection_string()

# Azure Application Insights にトレースを送信するように OpenTelemetry を構成します。
from azure.monitor.opentelemetry import configure_azure_monitor
configure_azure_monitor(connection_string=connection_string)

## OpenAI SDK 実行

In [ ]:
import os

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

completion = client.chat.completions.create(
    model="gpt-4.1",  # e.g. gpt-35-instant
    messages=[
        {
            "role": "user",
            "content": "君の名はなんていうの？",
        },
    ],
)
print(completion.to_json())

# AutoGen のトレース
AutoGenには、アプリケーションの実行に関する包括的な記録を収集するためのトレースと観測のサポートが組み込まれています。この機能は、デバッグ、パフォーマンス分析、そしてアプリケーションのフローを理解するのに役立ちます。

この機能はOpenTelemetryライブラリを活用しているため、OpenTelemetry と互換性のある任意のバックエンドを使用してトレースを収集および分析できます。

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4.1-mini",
    model="gpt-4.1-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMessageTermination  

# Create the final reviewer agent
writer_agent = AssistantAgent(
    "writer_agent",
    model_client=model_client,
    system_message="あなたはプロの小説家です。魅力的な文章を完結なタッチで書くことができます。",
)

In [ ]:
stream = writer_agent.run_stream(task="SEが異世界に転生して無双する異世界転生系短編小説を書いてください。")
await Console(stream)

## 独自のスパンを作成
複雑なアプリケーションを開発するときに、ビジネス ロジックとモデルを組み合わせたコードのセクションをキャプチャすると便利な場合があります。 OpenTelemetry では、スパンの概念を使用して、関心のあるセクションをキャプチャします。 独自のスパンの生成を開始するには、現在の トレーサー オブジェクトのインスタンスを取得します。

In [ ]:
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

In [ ]:
from typing import List

@tracer.start_as_current_span("write_and_critic_short_story")
async def write_and_critic_short_story():
    current_span = trace.get_current_span()
    # Set attributes for the current span
    current_span.set_attribute("operation.critic_count", 3)


    # Create the final reviewer agent
    writer_agent = AssistantAgent(
        "writer_agent",
        model_client=model_client,
        system_message="あなたはプロの小説家です。魅力的な文章を完結なタッチで書くことができます。",
    )


    # Create the final reviewer agent
    critic_agent1 = AssistantAgent(
        "critic_agent1",
        model_client=model_client,
        system_message="作成された文章を批判的な観点から評価し、改善点を提案すること。",
    )

    # Create the final reviewer agent
    critic_agent2 = AssistantAgent(
        "critic_agent2",
        model_client=model_client,
        system_message="あなたはリスクアドバイザーです。作成された文章のアイデア被りやSMS等で炎上しないかどうかを評価し、改善点を提案すること。",
    )

    # Create the final reviewer agent
    critic_agent3 = AssistantAgent(
        "critic_agent3",
        model_client=model_client,
        system_message="あなたは法学のスペシャリストです。作成された文章を法的観点から評価し、改善点を提案すること。",
    )

    # add agents to array
    critic_agents: List[AssistantAgent] = [
        writer_agent,
        critic_agent1,
        critic_agent2,
        critic_agent3,
    ]

    termination_condition = TextMessageTermination("critic_agent3") #3人目が発言したら終了
    critic_team = RoundRobinGroupChat(critic_agents, termination_condition=termination_condition)
            
    stream = critic_team.run_stream(task="SEが異世界に転生して無双する異世界転生系短編小説を書いてください。")
    await Console(stream)

    return

In [ ]:
await write_and_critic_short_story()